# Hyper-parameters tuning on HPC - NZ RSE 2020

This demo illustrates one simple way multiple ways to adapt a random search
strategy for hyper-parameters tuning to use HPC for the many parallel
computations involved.

In this example, we will rely on [Dask](https://dask.org) to do the heavy lifting,
distributing the parallel operations on SLURM jobs.

In [ ]:
import json
import warnings

import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

import joblib
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from dask_ml.model_selection import HyperbandSearchCV

import torch
import skorch
from hpc_for_datascience_demos.torch_models import SimpleCNN

Load MNIST data from [OpenML](https://www.openml.org/d/554).

In [ ]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = X / 255.0
y = y.astype(int)

This dataset contains images of digits. Here is a sample.

In [ ]:
_, axes = plt.subplots(1, 10, figsize=(12, 5))
for ax, digit in zip(axes, X):
    ax.imshow(digit.reshape(28, 28))
    ax.axis("off")

To keep this example code quick, let's use only a subset of the whole data set
as train and test sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, train_size=5000, test_size=10000, random_state=42
)

Fit a simple multi-layer perceptron neural net.

In [ ]:
%%time
mlp = MLPClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
y_pred = mlp.predict(X_test)
mlp_acc = accuracy_score(y_test, y_pred)
print(f"Baseline MLP test accuracy is {mlp_acc * 100:.2f}%.")

Tune hyper-parameters using a random search strategy.

In [ ]:
param_space = {
    "hidden_layer_sizes": st.randint(50, 200),
    "alpha": st.loguniform(1e-5, 1e-2),
    "learning_rate_init": st.loguniform(1e-4, 1e-1),
}
mlp_tuned = RandomizedSearchCV(
    MLPClassifier(random_state=42), param_space, n_iter=10, random_state=42, verbose=1
)

Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

- disabling the mechanism to write on disk when workers run out of memory,
- memory, CPUs, maximum time and number of workers per SLURM job,
- dask folders for log files and workers data.

All of these options can be set in configuration files, see [Dask configuration](https://docs.dask.org/en/latest/configuration.html)
and [Dask jobqueue configuration](https://jobqueue.dask.org/en/latest/configuration-setup.html)
for more information.

In [ ]:
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=4,
    processes=1,
    memory="4GiB",
    walltime="0-00:10",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)
client = Client(cluster)

In [ ]:
client

Spawn 10 workers and connect a client to be able use them.

In [ ]:
cluster.scale(n=10)
client.wait_for_workers(1)

Scikit-learn uses [Joblib](https://joblib.readthedocs.io) to parallelize
computations of many operations, including the randomized search on
hyper-parameters. If we configure Joblib to use Dask as a backend,
computations will be automatically scheduled and distributed on nodes of the
HPC platform.

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X_train, y_train]):
    mlp_tuned.fit(X_train, y_train)

Enjoy an optimized model :).

In [ ]:
y_pred_tuned = mlp_tuned.predict(X_test)
mlp_tuned_acc = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned MLP test accuracy is {mlp_tuned_acc * 100:.2f}%.")

In [ ]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_tuned.best_params_, indent=4)}")

This first approach requires very little changes but is far from optimal from
a ressource usage perspective. The [Dask-ML](https://ml.dask.org/) package
implements a more advanced algorithm, [Hyperband](https://arxiv.org/abs/1603.06560),
designed to better use a finite bugdet, using early stopping of bad
configurations. It relies on Scikit-learn API, assuming the estimator
implements the `partial_fit` method.

In [ ]:
mlp_hyper = HyperbandSearchCV(
    MLPClassifier(random_state=42),
    param_space,
    max_iter=200,
    aggressiveness=4,
    random_state=42,
)

In [ ]:
%%time
_ = mlp_hyper.fit(X_train, y_train, classes=np.unique(y))

In [ ]:
y_pred_hyper = mlp_hyper.predict(X_test)
mlp_hyper_acc = accuracy_score(y_test, y_pred_hyper)
print(f"MLP (Hyperband) test accuracy is {mlp_hyper_acc * 100:.2f}%.")

In [ ]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_hyper.best_params_, indent=4)}")

Now if we want to try some deep learning models trained with GPUs, we need to
start a new Dask cluster, requesting the right resources. First let's stop the
current cluster.

In [ ]:
cluster.close()
client.close()

Then start a new cluster, explicitly asking for GPUs.

In [ ]:
cluster = SLURMCluster(
    cores=4,
    processes=1,
    memory="6GiB",
    walltime="0-00:10",
    log_directory="../dask/logs",
    local_directory="../dask",
    job_extra=["--gres gpu:A100 --qos=testing"],  # passed to job script to request a GPU
    env_extra=["module load cuDNN/8.0.2.39-CUDA-11.0.2"], # run before worker start
)
client = Client(cluster)

Here we use an adaptive scaling strategy, asking Dask scheduler to start at
least one worker and letting it spawning on the fly more if needed.

In [ ]:
cluster.adapt(minimum=1, maximum=6)
client.wait_for_workers(1)

For the deep learning part, let's use a simple convolutional neural network
implemented using [Pytorch](https://pytorch.org/). We make use of [Skorch](https://github.com/skorch-dev/skorch)
to make it in a Scikit-learn compatible estimator.

In [ ]:
torch.manual_seed(0)
cnn = skorch.NeuralNetClassifier(
    module=SimpleCNN,
    module__input_dims=(28, 28),
    module__output_dim=len(np.unique(y)),
    module__n_chans=32,
    module__hidden_dim=100,
    module__dropout=0.5,
    optimizer=torch.optim.Adam,
    optimizer__lr=1e-3,
    device="cuda",
)

Thanks to Skorch, our model is compatible with Scikit-learn API, and thus can
be used with Dask-ML meta-estimators. Hence we will use Hyberband to search
for the best hyper-parameters.

In [ ]:
param_space = {
    "module__n_chans": st.randint(10, 64),
    "module__hidden_dim": st.randint(50, 200),
    "module__dropout": st.uniform(),
    "optimizer__lr": st.loguniform(1e-4, 1e-1),
}
mlp_torch = HyperbandSearchCV(cnn, param_space, max_iter=20, random_state=42)

In [ ]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mlp_torch.fit(X_train.astype(np.float32), y_train)

In [ ]:
y_pred_torch = mlp_torch.predict(X_test.astype(np.float32))
mlp_torch_acc = accuracy_score(y_test, y_pred_torch)
print(f"CNN (PyTorch & Hyperband) test accuracy is {mlp_torch_acc * 100:.2f}%.")

In [ ]:
print(f"Best hyper-parameters:\n{json.dumps(mlp_torch.best_params_, indent=4)}")